<a href="https://colab.research.google.com/github/satyaabdul/Project/blob/main/Capstone_Project_Customer_Support_Classification_and_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install library datasets
!pip install datasets -q

In [2]:
#Install library replicate
!pip install langchain_community -q

In [3]:
#Import library
import kagglehub
from datasets import load_dataset
import pandas as pd
from langchain_community.llms import Replicate
from google.colab import userdata
import os

In [4]:
#Set dan ambil API Token dari Google Colab Secrets
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

In [5]:
#Set model IBM Granite via Replicate
model = "ibm-granite/granite-3.3-8b-instruct"
granite = Replicate(
    model=model,
    replicate_api_token=api_token,
)

In [6]:
#Download dataset
path = kagglehub.dataset_download("thoughtvector/customer-support-on-twitter")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/customer-support-on-twitter


In [7]:
#Masukan dalam dataframe
file_path = f"{path}/sample.csv"
df = pd.read_csv(file_path)
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [8]:
#Ambil sample
samples = df[df["inbound"] == True]["text"].dropna().sample(3, random_state=42).tolist()
print(samples)

['First flight for long time with @British_Airways. Now over one 1h delay for the short jump FRA-LCY and NO one here to provide status updates', '@HPSupport Problem with printhead  Serial# TH536D1HN #hpcswus #hpprnt', 'They reschedule my shit for tomorrow https://t.co/RsvZcT982t']


In [9]:
#Set prompt untuk IBM Granite
prompt = """
Please perform the following two tasks for each customer message:

1. Classify the message into one of the following categories: Technical, Payment, Product Request, Other.
   If possible, include a short label in parentheses (e.g., delay issue, login failure, refund request).
2. Generate a brief summary of the message content in no more than one short sentence.

Respond using the following format for each customer:

Customer 1:
Category: <your classification>
Summary: <your summary>

"""

#Gabungkan jawaban
for i, text in enumerate(samples):
    prompt += f"\nCustomer {i+1}: {text}"


In [10]:
#Kirim prompt ke Granite
response = granite.invoke(prompt)

#Tampilkan hasil
print(response)

Customer 1:  
Category: Technical (flight delay)  
Summary: The customer expresses frustration over a significant delay in their short British Airways flight from FRA to LCY without any updates.

Customer 2:  
Category: Technical  
Summary: The customer reports an issue with the printhead of their HP printer, providing the serial number for reference.

Customer 3:  
Category: Other (rescheduled appointment)  
Summary: The customer's appointment has been rescheduled to the following day, as indicated by a shared link to further details.


In [11]:
#Tetapkan parameter
parameters = {
    "top_k": 0,
    "top_p": 0.9,
    "max_tokens": 300,
    "min_tokens": 0,
    "random_seed": 42,
    "repetition_penalty": 1.1,
    "stopping_criteria": "length",
    "stopping_sequence": None
}


In [12]:
#Kirim prompt ke Granite dengan parameter
response_param = granite.invoke(prompt, **parameters)

#Tampilkan hasil denan parameter
print(response_param)

Customer 1:  
Category: Technical (flight delay)  
Summary: The customer expresses frustration over a significant delay in their short British Airways flight from FRA to LCY without any status updates.

Customer 2:  
Category: Technical  
Summary: The customer reports an issue with the printhead of their HP printer, providing the serial number for reference.

Customer 3:  
Category: Reschedule (appointment)  
Summary: The customer's appointment was rescheduled to the following day, as indicated by the linked tweet or image.
